# Part 2 - Mapping Yelp Search Results

## Obective

- For this CodeAlong, we will be working with the Yelp API results from last class. 
- You will load in the .csv.gz of your yelp results and prepare the data for visualization.
- You will use Plotly Express to create an interactive map with all of the results.

## Tools You Will Use
- Part 1:
    - Yelp API:
        - Getting Started: 
            - https://www.yelp.com/developers/documentation/v3/get_started

    - `YelpAPI` python package
        -  "YelpAPI": https://github.com/gfairchild/yelpapi
- Part 2:

    - Plotly Express: https://plotly.com/python/getting-started/
        - With Mapbox API: https://www.mapbox.com/
        - `px.scatter_mapbox` [Documentation](https://plotly.com/python/scattermapbox/): 




### Applying Code From
- [Advanced Transformations with Pandas - Part 1](https://login.codingdojo.com/m/376/12529/88086)
- [Advanced Transformations with Pandas - Part 2](https://login.codingdojo.com/m/376/12529/88088)

### Goal

- We want to create a map with every restaurant plotted as a scatter plot with detailed information that appears when we hover over a business
- We will use plotly express's `px.scatter_mapbox` function to accomplish this.
    - https://plotly.com/python/scattermapbox/
    
    - We will need a Mapbox API token for some of the options:
        - https://studio.mapbox.com/
    

# Loading Data from Part 1

In [61]:
## Plotly is not included in your dojo-env
!pip install plotly

In [62]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import json

## importing plotly 
import plotly.express as px

In [63]:
## Load in csv.gz
df = pd.read_csv('Data/Seattle-pizza.csv.gz')
df.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,t-Z9bvmlgUyDtomGmttrUQ,guerilla-pizza-kitchen-seattle,Guerilla Pizza Kitchen,https://s3-media4.fl.yelpcdn.com/bphoto/uMrdEj...,False,https://www.yelp.com/biz/guerilla-pizza-kitche...,1,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",5.0,"{'latitude': 47.608127, 'longitude': -122.302435}",[],"{'address1': '2300 E Cherry St', 'address2': '...",NaN,NaN,749.173113,NaN
1,Wi6LFkjIausYj277ru6pqg,humble-pie-seattle,Humble Pie,https://s3-media4.fl.yelpcdn.com/bphoto/A08GOZ...,False,https://www.yelp.com/biz/humble-pie-seattle?ad...,350,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 47.5976491915013, 'longitude': -1...",['delivery'],"{'address1': '525 Rainier Ave S', 'address2': ...",1.206330e+10,(206) 329-5133,2166.834011,$$
2,EwrAoy7XbX8nfemZdDvupA,cornelly-seattle,Cornelly,https://s3-media4.fl.yelpcdn.com/bphoto/HlYYBT...,False,https://www.yelp.com/biz/cornelly-seattle?adju...,88,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 47.624577, 'longitude': -122.325577}","['pickup', 'delivery']","{'address1': '601 Summit Ave E', 'address2': '...",NaN,NaN,2578.701393,NaN
3,FVbwpNA1uZEGiM02N4XtUg,blotto-seattle,Blotto,https://s3-media3.fl.yelpcdn.com/bphoto/Q4Y1Cw...,False,https://www.yelp.com/biz/blotto-seattle?adjust...,32,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 47.61849, 'longitude': -122.31664}",[],"{'address1': '1830 12th Ave', 'address2': '', ...",1.206403e+10,(206) 403-1809,1686.862176,NaN
4,M9xzvwgK58T0w7wvXedvuQ,hot-mamas-pizza-seattle,Hot Mama's Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/LEL1qj...,False,https://www.yelp.com/biz/hot-mamas-pizza-seatt...,937,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 47.615379179632, 'longitude': -12...",['delivery'],"{'address1': '700 E Pine St', 'address2': '', ...",1.206323e+10,(206) 322-6444,2080.359405,$


## Required Preprocessing 

- 1. We need to get the latitude and longitude for each business as separate columns.
- We also want to be able to show the restaurants:
    - name,
    - price range
    - address
    - and if they do delivery or takeout.

### Separating Latitude and Longitude

In [64]:
## use .apply pd.Series to convert a dict to columns
df['coordinates'].apply(pd.Series)

,0
0,"{'latitude': 47.608127, 'longitude': -122.302435}"
1,"{'latitude': 47.5976491915013, 'longitude': -1..."
2,"{'latitude': 47.624577, 'longitude': -122.325577}"
3,"{'latitude': 47.61849, 'longitude': -122.31664}"
4,"{'latitude': 47.615379179632, 'longitude': -12..."
...,...
835,"{'latitude': 47.6179959197781, 'longitude': -1..."
836,"{'latitude': 47.6157033283809, 'longitude': -1..."
837,"{'latitude': 47.61252119845868, 'longitude': -..."
838,"{'latitude': 47.676215, 'longitude': -122.205084}"


- Why didn't that work???

In [65]:
## slice out a single test coordinate
test_coord = df.loc[0,'coordinates']
test_coord

"{'latitude': 47.608127, 'longitude': -122.302435}"

In [66]:
type(test_coord)

str

- Its not a dictionary anymore!!! ??
    - CSV files cant store iterables (lists, dictionaries) so they get converted to strings.

### Fixing the String-Dictionaries

- The json module has another version of load and dump called `json.loads` and `json.dumps`
    - These are designed to process STRINGS instead of files. 
    
- If we use `json.loads` we can convert our string dictionary into an actual dictionary. 

In [67]:
## Use json.loads on the test coordinate
json.loads(test_coord)

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)

- JSON requires double quotes!

In [68]:
## replace single ' with " 
test_coord =test_coord.replace("'",'"')
test_coord

'{"latitude": 47.608127, "longitude": -122.302435}'

In [69]:
## Use json.loads on the test coordinate, again
json.loads(test_coord)

{'latitude': 47.608127, 'longitude': -122.302435}

### Now, how can we apply this same process to the entire column??

In [70]:
## replace ' with " (entire column)
df['coordinates'] = df['coordinates'].str.replace("'",'"')

## apply json.loads
df['coordinates'] = df['coordinates'].apply(json.loads)

In [71]:
## slice out a single test coordinate
test_coord = df.loc[0,'coordinates']
type(test_coord)

dict

### Using Apply with pd.Series to convert a dictionary column into multiple columns

In [72]:
## use .apply pd.Series to convert a dict to columns
df['coordinates'].apply(pd.Series)

,latitude,longitude
0,47.608127,-122.302435
1,47.597649,-122.313306
2,47.624577,-122.325577
3,47.618490,-122.316640
4,47.615379,-122.323231
...,...,...
835,47.617996,-122.200456
836,47.615703,-122.200735
837,47.612521,-122.201333
838,47.676215,-122.205084


In [73]:
## Concatenate the 2 new columns and drop the original.
df = pd.concat([df,df['coordinates'].apply(pd.Series)],axis = 1)
df=df.drop(columns='coordinates')
df.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,location,phone,display_phone,distance,price,latitude,longitude
0,t-Z9bvmlgUyDtomGmttrUQ,guerilla-pizza-kitchen-seattle,Guerilla Pizza Kitchen,https://s3-media4.fl.yelpcdn.com/bphoto/uMrdEj...,False,https://www.yelp.com/biz/guerilla-pizza-kitche...,1,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",5.0,[],"{'address1': '2300 E Cherry St', 'address2': '...",NaN,NaN,749.173113,NaN,47.608127,-122.302435
1,Wi6LFkjIausYj277ru6pqg,humble-pie-seattle,Humble Pie,https://s3-media4.fl.yelpcdn.com/bphoto/A08GOZ...,False,https://www.yelp.com/biz/humble-pie-seattle?ad...,350,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,['delivery'],"{'address1': '525 Rainier Ave S', 'address2': ...",1.206330e+10,(206) 329-5133,2166.834011,$$,47.597649,-122.313306
2,EwrAoy7XbX8nfemZdDvupA,cornelly-seattle,Cornelly,https://s3-media4.fl.yelpcdn.com/bphoto/HlYYBT...,False,https://www.yelp.com/biz/cornelly-seattle?adju...,88,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"['pickup', 'delivery']","{'address1': '601 Summit Ave E', 'address2': '...",NaN,NaN,2578.701393,NaN,47.624577,-122.325577
3,FVbwpNA1uZEGiM02N4XtUg,blotto-seattle,Blotto,https://s3-media3.fl.yelpcdn.com/bphoto/Q4Y1Cw...,False,https://www.yelp.com/biz/blotto-seattle?adjust...,32,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,[],"{'address1': '1830 12th Ave', 'address2': '', ...",1.206403e+10,(206) 403-1809,1686.862176,NaN,47.618490,-122.316640
4,M9xzvwgK58T0w7wvXedvuQ,hot-mamas-pizza-seattle,Hot Mama's Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/LEL1qj...,False,https://www.yelp.com/biz/hot-mamas-pizza-seatt...,937,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,['delivery'],"{'address1': '700 E Pine St', 'address2': '', ...",1.206323e+10,(206) 322-6444,2080.359405,$,47.615379,-122.323231


## Creating a Simple Map

### Register for MapBox API

Mapbox API: https://www.mapbox.com/

In [74]:
## Load in mapbox api credentials from .secret
with open('/Users/purvikansara/.secret/mapbox.json') as f:
    login = json.load(f)
login.keys()

dict_keys(['client-id', 'api-key'])

- Use the plotly express `set_maptbox_acccess_token` function

In [75]:
## set mapbox token
px.set_mapbox_access_token(login['api-key'])

In [76]:
## use scatter_mapbox for M.V.P map
px.scatter_mapbox(df,lat = 'latitude',lon = 'longitude', mapbox_style='open-street-map')

### Adding Hover Data

- We want to show the restaurants:
    - name
    - price range
    - address
    - and if they do delivery or takeout.
    
    
- We can use the `hover_name` and `hover_data` arguments for `px.scatter_mapbox` to add this info!

In [77]:

df.head(2)

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,location,phone,display_phone,distance,price,latitude,longitude
0,t-Z9bvmlgUyDtomGmttrUQ,guerilla-pizza-kitchen-seattle,Guerilla Pizza Kitchen,https://s3-media4.fl.yelpcdn.com/bphoto/uMrdEj...,False,https://www.yelp.com/biz/guerilla-pizza-kitche...,1,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",5.0,[],"{'address1': '2300 E Cherry St', 'address2': '...",NaN,NaN,749.173113,NaN,47.608127,-122.302435
1,Wi6LFkjIausYj277ru6pqg,humble-pie-seattle,Humble Pie,https://s3-media4.fl.yelpcdn.com/bphoto/A08GOZ...,False,https://www.yelp.com/biz/humble-pie-seattle?ad...,350,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,['delivery'],"{'address1': '525 Rainier Ave S', 'address2': ...",1.206330e+10,(206) 329-5133,2166.834011,$$,47.597649,-122.313306


In [78]:
## add hover_name (name) and hover_data for price,rating,location
px.scatter_mapbox(df, lat='latitude',lon='longitude',
                  mapbox_style='open-street-map', hover_name='name',
                 hover_data=['price','rating','location'])


In [79]:
## add hover_name (name) and hover_data for price,rating,location
px.scatter_mapbox(df, lat='latitude',lon='longitude',color="rating",color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=10,
                   hover_name='name',
                 hover_data=['price','rating','location'])

### Fixing the Location Column

In [80]:
## slice out a test address
test_addr = df.loc[0, 'location']
test_addr

"{'address1': '2300 E Cherry St', 'address2': '', 'address3': None, 'city': 'Seattle', 'zip_code': '98122', 'country': 'US', 'state': 'WA', 'display_address': ['2300 E Cherry St', 'Seattle, WA 98122']}"

> Also a string-dictionary...

In [81]:
## replace ' with "
df['location'] = df['location'].str.replace("'", '"')
df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,location,phone,display_phone,distance,price,latitude,longitude
0,t-Z9bvmlgUyDtomGmttrUQ,guerilla-pizza-kitchen-seattle,Guerilla Pizza Kitchen,https://s3-media4.fl.yelpcdn.com/bphoto/uMrdEj...,False,https://www.yelp.com/biz/guerilla-pizza-kitche...,1,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",5.0,[],"{""address1"": ""2300 E Cherry St"", ""address2"": ""...",NaN,NaN,749.173113,NaN,47.608127,-122.302435
1,Wi6LFkjIausYj277ru6pqg,humble-pie-seattle,Humble Pie,https://s3-media4.fl.yelpcdn.com/bphoto/A08GOZ...,False,https://www.yelp.com/biz/humble-pie-seattle?ad...,350,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,['delivery'],"{""address1"": ""525 Rainier Ave S"", ""address2"": ...",1.206330e+10,(206) 329-5133,2166.834011,$$,47.597649,-122.313306
2,EwrAoy7XbX8nfemZdDvupA,cornelly-seattle,Cornelly,https://s3-media4.fl.yelpcdn.com/bphoto/HlYYBT...,False,https://www.yelp.com/biz/cornelly-seattle?adju...,88,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"['pickup', 'delivery']","{""address1"": ""601 Summit Ave E"", ""address2"": ""...",NaN,NaN,2578.701393,NaN,47.624577,-122.325577
3,FVbwpNA1uZEGiM02N4XtUg,blotto-seattle,Blotto,https://s3-media3.fl.yelpcdn.com/bphoto/Q4Y1Cw...,False,https://www.yelp.com/biz/blotto-seattle?adjust...,32,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,[],"{""address1"": ""1830 12th Ave"", ""address2"": """", ...",1.206403e+10,(206) 403-1809,1686.862176,NaN,47.618490,-122.316640
4,M9xzvwgK58T0w7wvXedvuQ,hot-mamas-pizza-seattle,Hot Mama's Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/LEL1qj...,False,https://www.yelp.com/biz/hot-mamas-pizza-seatt...,937,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,['delivery'],"{""address1"": ""700 E Pine St"", ""address2"": """", ...",1.206323e+10,(206) 322-6444,2080.359405,$,47.615379,-122.323231
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835,H_gsS9J920u_hK7jekobqw,suite-bellevue-4,Suite,https://s3-media1.fl.yelpcdn.com/bphoto/jdYMfv...,False,https://www.yelp.com/biz/suite-bellevue-4?adju...,225,"[{'alias': 'lounges', 'title': 'Lounges'}, {'a...",3.0,['delivery'],"{""address1"": ""10500 NE 8th St"", ""address2"": ""S...",1.425680e+10,(425) 679-6951,7158.825860,$$,47.617996,-122.200456
836,rcDbUcs83gvoRxhaTmFfnQ,cypress-lounge-and-wine-bar-bellevue,Cypress Lounge & Wine Bar,https://s3-media2.fl.yelpcdn.com/bphoto/DsFhIb...,False,https://www.yelp.com/biz/cypress-lounge-and-wi...,64,"[{'alias': 'wine_bars', 'title': 'Wine Bars'},...",3.5,[],"{""address1"": ""600 Bellevue Way NE"", ""address2""...",1.425638e+10,(425) 638-1000,7123.208902,$$,47.615703,-122.200735
837,oniQ68t6VR1naVtUXmfETQ,pokeworks-bellevue-2,Pokeworks,https://s3-media1.fl.yelpcdn.com/bphoto/DqSOK8...,False,https://www.yelp.com/biz/pokeworks-bellevue-2?...,270,"[{'alias': 'poke', 'title': 'Poke'}, {'alias':...",4.0,"['pickup', 'delivery']","{""address1"": ""222 Bellevue Way NE"", ""address2""...",1.425214e+10,(425) 214-1182,7072.982860,$$,47.612521,-122.201333
838,8S3qU0TGMJDyj4KHBmxlaw,pho-tai-kirkland,Pho Tai,https://s3-media3.fl.yelpcdn.com/bphoto/tnvGkA...,False,https://www.yelp.com/biz/pho-tai-kirkland?adju...,189,"[{'alias': 'vietnamese', 'title': 'Vietnamese'...",3.5,"['pickup', 'delivery']","{""address1"": ""147 Park Ln"", ""address2"": """", ""a...",1.425577e+10,(425) 576-5407,9765.353049,$$,47.676215,-122.205084


In [82]:
# ## apply json.loads
df['location'] = df['location'].apply(json.loads)
df

JSONDecodeError: Expecting value: line 1 column 62 (char 61)

> Ruh roh....

- Hmm, let's slice out a test_address again and let's write a function to accomplish this instead.
    - We can use try and except in our function to get around the errors.

In [83]:
type(df.loc[0,'location'])

str

### Fixing Addresses - with a custom function


In [84]:
## slice out test address 
test_addr = df.loc[0, 'location']
test_addr

'{"address1": "2300 E Cherry St", "address2": "", "address3": None, "city": "Seattle", "zip_code": "98122", "country": "US", "state": "WA", "display_address": ["2300 E Cherry St", "Seattle, WA 98122"]}'

In [85]:
## write a function to just run json.loads on the address
def fix_address(test_addr):
    try:
        return json.loads(test_addr)
    except:
        return 'Error'

In [86]:
## test applying our function
df['location'].apply(fix_address)

0                                                  Error
1      {'address1': '525 Rainier Ave S', 'address2': ...
2                                                  Error
3                                                  Error
4      {'address1': '700 E Pine St', 'address2': '', ...
                             ...                        
835    {'address1': '10500 NE 8th St', 'address2': 'S...
836    {'address1': '600 Bellevue Way NE', 'address2'...
837                                                Error
838    {'address1': '147 Park Ln', 'address2': '', 'a...
839                                                Error
Name: location, Length: 840, dtype: object

- It worked! Now let's save this as a new column (display_location),
and then let's investigate the businesses that had an "ERROR".

In [87]:
### save a new display_location column using our function
df['display_location'] = df['location'].apply(fix_address)

In [88]:
## filter for businesses with display_location == "ERROR"
errors = df[df['display_location']=='Error']
errors

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,location,phone,display_phone,distance,price,latitude,longitude,display_location
0,t-Z9bvmlgUyDtomGmttrUQ,guerilla-pizza-kitchen-seattle,Guerilla Pizza Kitchen,https://s3-media4.fl.yelpcdn.com/bphoto/uMrdEj...,False,https://www.yelp.com/biz/guerilla-pizza-kitche...,1,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",5.0,[],"{""address1"": ""2300 E Cherry St"", ""address2"": ""...",NaN,NaN,749.173113,NaN,47.608127,-122.302435,Error
2,EwrAoy7XbX8nfemZdDvupA,cornelly-seattle,Cornelly,https://s3-media4.fl.yelpcdn.com/bphoto/HlYYBT...,False,https://www.yelp.com/biz/cornelly-seattle?adju...,88,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"['pickup', 'delivery']","{""address1"": ""601 Summit Ave E"", ""address2"": ""...",NaN,NaN,2578.701393,NaN,47.624577,-122.325577,Error
3,FVbwpNA1uZEGiM02N4XtUg,blotto-seattle,Blotto,https://s3-media3.fl.yelpcdn.com/bphoto/Q4Y1Cw...,False,https://www.yelp.com/biz/blotto-seattle?adjust...,32,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,[],"{""address1"": ""1830 12th Ave"", ""address2"": """", ...",1.206403e+10,(206) 403-1809,1686.862176,NaN,47.618490,-122.316640,Error
6,IrohtoYjnAR_vc6w6CRCxA,moto-seattle,Moto,https://s3-media2.fl.yelpcdn.com/bphoto/FBmvjy...,False,https://www.yelp.com/biz/moto-seattle?adjust_c...,111,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",5.0,['delivery'],"{""address1"": ""4526 42nd Ave SW"", ""address2"": ""...",1.206421e+10,(206) 420-8880,8784.075722,NaN,47.562070,-122.385090,Error
8,wi-mbKEz4Ha1YYaOB8zHnw,kōbo-pizza-seattle,Kōbo Pizza,https://s3-media1.fl.yelpcdn.com/bphoto/6et4T9...,False,https://www.yelp.com/biz/k%C5%8Dbo-pizza-seatt...,1,"[{'alias': 'pizza', 'title': 'Pizza'}]",5.0,[],"{""address1"": ""714 E Pike St"", ""address2"": None...",1.253693e+10,(253) 693-0207,2025.139396,NaN,47.614400,-122.322710,Error
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
825,3cLNDJ2vI29LWNvU7vVIuQ,capital-one-café-bellevue-2,Capital One Café,https://s3-media1.fl.yelpcdn.com/bphoto/-o6eAW...,False,https://www.yelp.com/biz/capital-one-caf%C3%A9...,38,"[{'alias': 'banks', 'title': 'Banks & Credit U...",4.0,[],"{""address1"": ""400 Bellevue Way NE"", ""address2""...",1.425372e+10,(425) 372-0250,7071.129218,$,47.614080,-122.201363,Error
826,Nihv2RfuTZ4TI8BQl3anmA,qfc-kirkland-4,QFC,https://s3-media2.fl.yelpcdn.com/bphoto/NYz4rH...,False,https://www.yelp.com/biz/qfc-kirkland-4?adjust...,30,"[{'alias': 'grocery', 'title': 'Grocery'}]",3.0,[],"{""address1"": ""457 Central Way"", ""address2"": """"...",1.425827e+10,(425) 827-2205,10212.263314,$$,47.678048,-122.199308,Error
834,AsgtzyqpdjuPyjaEUVcUXA,jewel-of-india-seattle-3,Jewel Of India,https://s3-media2.fl.yelpcdn.com/bphoto/qJGMnl...,False,https://www.yelp.com/biz/jewel-of-india-seattl...,405,"[{'alias': 'indpak', 'title': 'Indian'}]",3.5,"['pickup', 'delivery']","{""address1"": ""4735 University Way NE"", ""addres...",1.206524e+10,(206) 523-5275,5825.461180,$$,47.664103,-122.313426,Error
837,oniQ68t6VR1naVtUXmfETQ,pokeworks-bellevue-2,Pokeworks,https://s3-media1.fl.yelpcdn.com/bphoto/DqSOK8...,False,https://www.yelp.com/biz/pokeworks-bellevue-2?...,270,"[{'alias': 'poke', 'title': 'Poke'}, {'alias':...",4.0,"['pickup', 'delivery']","{""address1"": ""222 Bellevue Way NE"", ""address2""...",1.425214e+10,(425) 214-1182,7072.982860,$$,47.612521,-122.201333,Error


In [89]:
## slice out a new test address and inspect
test_addr = df.loc[839, 'location']
test_addr

'{"address1": "1015 108th Ave NE", "address2": "", "address3": None, "city": "Bellevue", "zip_code": "98004", "country": "US", "state": "WA", "display_address": ["1015 108th Ave NE", "Bellevue, WA 98004"]}'

In [90]:
## slice out a new test address and inspect
test_addr = df.loc[errors.index[1], 'location']
test_addr

'{"address1": "601 Summit Ave E", "address2": "", "address3": None, "city": "Seattle", "zip_code": "98102", "country": "US", "state": "WA", "display_address": ["601 Summit Ave E", "Seattle, WA 98102"]}'

> After some more investigation, we would find a few issues with these "ERROR" rows.
1. They contained None.
2. They contained an apostrophe in the name.
3. ...?

### Possible Fixes (if we care to/have the time)


- Use Regular Expressions to find an fix the display addresses with "'" in them
- Use string split to split on the word display address.
    - Then use string methods to clean up

### Moving Forward without those rows (for now)

In [91]:
## remove any rows where display_location == 'ERROR'
df = df.loc[df['display_location'] != 'Error']
df.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,location,phone,display_phone,distance,price,latitude,longitude,display_location
1,Wi6LFkjIausYj277ru6pqg,humble-pie-seattle,Humble Pie,https://s3-media4.fl.yelpcdn.com/bphoto/A08GOZ...,False,https://www.yelp.com/biz/humble-pie-seattle?ad...,350,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,['delivery'],"{""address1"": ""525 Rainier Ave S"", ""address2"": ...",1.206330e+10,(206) 329-5133,2166.834011,$$,47.597649,-122.313306,"{'address1': '525 Rainier Ave S', 'address2': ..."
4,M9xzvwgK58T0w7wvXedvuQ,hot-mamas-pizza-seattle,Hot Mama's Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/LEL1qj...,False,https://www.yelp.com/biz/hot-mamas-pizza-seatt...,937,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,['delivery'],"{""address1"": ""700 E Pine St"", ""address2"": """", ...",1.206323e+10,(206) 322-6444,2080.359405,$,47.615379,-122.323231,"{'address1': '700 E Pine St', 'address2': '', ..."
5,fxyHWmfzcdjImgQ_UYHoTw,bar-cotto-seattle,Bar Cotto,https://s3-media3.fl.yelpcdn.com/bphoto/07DoPF...,False,https://www.yelp.com/biz/bar-cotto-seattle?adj...,335,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.5,['delivery'],"{""address1"": ""1546 15th Ave"", ""address2"": """", ...",1.206839e+10,(206) 838-8081,1292.526796,$$,47.614693,-122.312764,"{'address1': '1546 15th Ave', 'address2': '', ..."
7,et4eWK49x5od4lfqCukG6w,central-pizza-seattle,Central Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/DE4G4e...,False,https://www.yelp.com/biz/central-pizza-seattle...,196,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.0,"['pickup', 'delivery']","{""address1"": ""2901 S Jackson St"", ""address2"": ...",1.206603e+10,(206) 602-6333,1571.867438,$$,47.598976,-122.294685,"{'address1': '2901 S Jackson St', 'address2': ..."
9,cAFXng3Gr1cBCFkP1VIxng,the-independent-pizzeria-seattle-2,The Independent Pizzeria,https://s3-media4.fl.yelpcdn.com/bphoto/4gIuis...,False,https://www.yelp.com/biz/the-independent-pizze...,179,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,['delivery'],"{""address1"": ""4235 E Madison St"", ""address2"": ...",1.206861e+10,(206) 860-6110,2897.252844,$$,47.636044,-122.277377,"{'address1': '4235 E Madison St', 'address2': ..."


- We want the "display_address" key from the "display_location" dictionaries.
- We could use a .apply and a lamda to slice out the desired key.

In [92]:
## use apply and lambda to slice correct key
df['display_address'] = df['display_location'].apply(lambda x: x['display_address'])
df.head()

/var/folders/td/cybzgn316hq8r5cn05lb0y2m0000gn/T/ipykernel_3854/2291832266.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,location,phone,display_phone,distance,price,latitude,longitude,display_location,display_address
1,Wi6LFkjIausYj277ru6pqg,humble-pie-seattle,Humble Pie,https://s3-media4.fl.yelpcdn.com/bphoto/A08GOZ...,False,https://www.yelp.com/biz/humble-pie-seattle?ad...,350,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,['delivery'],"{""address1"": ""525 Rainier Ave S"", ""address2"": ...",1.206330e+10,(206) 329-5133,2166.834011,$$,47.597649,-122.313306,"{'address1': '525 Rainier Ave S', 'address2': ...","[525 Rainier Ave S, Seattle, WA 98144]"
4,M9xzvwgK58T0w7wvXedvuQ,hot-mamas-pizza-seattle,Hot Mama's Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/LEL1qj...,False,https://www.yelp.com/biz/hot-mamas-pizza-seatt...,937,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,['delivery'],"{""address1"": ""700 E Pine St"", ""address2"": """", ...",1.206323e+10,(206) 322-6444,2080.359405,$,47.615379,-122.323231,"{'address1': '700 E Pine St', 'address2': '', ...","[700 E Pine St, Seattle, WA 98122]"
5,fxyHWmfzcdjImgQ_UYHoTw,bar-cotto-seattle,Bar Cotto,https://s3-media3.fl.yelpcdn.com/bphoto/07DoPF...,False,https://www.yelp.com/biz/bar-cotto-seattle?adj...,335,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.5,['delivery'],"{""address1"": ""1546 15th Ave"", ""address2"": """", ...",1.206839e+10,(206) 838-8081,1292.526796,$$,47.614693,-122.312764,"{'address1': '1546 15th Ave', 'address2': '', ...","[1546 15th Ave, Seattle, WA 98122]"
7,et4eWK49x5od4lfqCukG6w,central-pizza-seattle,Central Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/DE4G4e...,False,https://www.yelp.com/biz/central-pizza-seattle...,196,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.0,"['pickup', 'delivery']","{""address1"": ""2901 S Jackson St"", ""address2"": ...",1.206603e+10,(206) 602-6333,1571.867438,$$,47.598976,-122.294685,"{'address1': '2901 S Jackson St', 'address2': ...","[2901 S Jackson St, Seattle, WA 98144]"
9,cAFXng3Gr1cBCFkP1VIxng,the-independent-pizzeria-seattle-2,The Independent Pizzeria,https://s3-media4.fl.yelpcdn.com/bphoto/4gIuis...,False,https://www.yelp.com/biz/the-independent-pizze...,179,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,['delivery'],"{""address1"": ""4235 E Madison St"", ""address2"": ...",1.206861e+10,(206) 860-6110,2897.252844,$$,47.636044,-122.277377,"{'address1': '4235 E Madison St', 'address2': ...","[4235 E Madison St, Seattle, WA 98112]"


- Almost done! We want to convert display_address to a string instead a list of strings.
- We can use the string method .join to do so!

In [93]:
## slice out a test_address
test_addr = df.loc[9, 'display_address']
test_addr

['4235 E Madison St', 'Seattle, WA 98112']

In [94]:

## test using .join with a "\n"
'\n'.join(test_addr)

'4235 E Madison St\nSeattle, WA 98112'

In [95]:
## apply the join to every row with a lambda
## apply the join to every row with a lambda
df['Address'] = df['display_address'].apply(lambda x: '\n'.join(x))
df

/var/folders/td/cybzgn316hq8r5cn05lb0y2m0000gn/T/ipykernel_3854/396965205.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,location,phone,display_phone,distance,price,latitude,longitude,display_location,display_address,Address
1,Wi6LFkjIausYj277ru6pqg,humble-pie-seattle,Humble Pie,https://s3-media4.fl.yelpcdn.com/bphoto/A08GOZ...,False,https://www.yelp.com/biz/humble-pie-seattle?ad...,350,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,['delivery'],"{""address1"": ""525 Rainier Ave S"", ""address2"": ...",1.206330e+10,(206) 329-5133,2166.834011,$$,47.597649,-122.313306,"{'address1': '525 Rainier Ave S', 'address2': ...","[525 Rainier Ave S, Seattle, WA 98144]","525 Rainier Ave S\nSeattle, WA 98144"
4,M9xzvwgK58T0w7wvXedvuQ,hot-mamas-pizza-seattle,Hot Mama's Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/LEL1qj...,False,https://www.yelp.com/biz/hot-mamas-pizza-seatt...,937,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,['delivery'],"{""address1"": ""700 E Pine St"", ""address2"": """", ...",1.206323e+10,(206) 322-6444,2080.359405,$,47.615379,-122.323231,"{'address1': '700 E Pine St', 'address2': '', ...","[700 E Pine St, Seattle, WA 98122]","700 E Pine St\nSeattle, WA 98122"
5,fxyHWmfzcdjImgQ_UYHoTw,bar-cotto-seattle,Bar Cotto,https://s3-media3.fl.yelpcdn.com/bphoto/07DoPF...,False,https://www.yelp.com/biz/bar-cotto-seattle?adj...,335,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.5,['delivery'],"{""address1"": ""1546 15th Ave"", ""address2"": """", ...",1.206839e+10,(206) 838-8081,1292.526796,$$,47.614693,-122.312764,"{'address1': '1546 15th Ave', 'address2': '', ...","[1546 15th Ave, Seattle, WA 98122]","1546 15th Ave\nSeattle, WA 98122"
7,et4eWK49x5od4lfqCukG6w,central-pizza-seattle,Central Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/DE4G4e...,False,https://www.yelp.com/biz/central-pizza-seattle...,196,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.0,"['pickup', 'delivery']","{""address1"": ""2901 S Jackson St"", ""address2"": ...",1.206603e+10,(206) 602-6333,1571.867438,$$,47.598976,-122.294685,"{'address1': '2901 S Jackson St', 'address2': ...","[2901 S Jackson St, Seattle, WA 98144]","2901 S Jackson St\nSeattle, WA 98144"
9,cAFXng3Gr1cBCFkP1VIxng,the-independent-pizzeria-seattle-2,The Independent Pizzeria,https://s3-media4.fl.yelpcdn.com/bphoto/4gIuis...,False,https://www.yelp.com/biz/the-independent-pizze...,179,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,['delivery'],"{""address1"": ""4235 E Madison St"", ""address2"": ...",1.206861e+10,(206) 860-6110,2897.252844,$$,47.636044,-122.277377,"{'address1': '4235 E Madison St', 'address2': ...","[4235 E Madison St, Seattle, WA 98112]","4235 E Madison St\nSeattle, WA 98112"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
832,IelbaEgtvDbAFucJWLUWRw,marketplace-cafe-bellevue-7,Marketplace Cafe,https://s3-media1.fl.yelpcdn.com/bphoto/JxhPMe...,False,https://www.yelp.com/biz/marketplace-cafe-bell...,126,"[{'alias': 'newamerican', 'title': 'American (...",3.0,['delivery'],"{""address1"": ""100 Bellevue Sq"", ""address2"": """"...",1.425647e+10,(425) 646-5006,6891.374266,$$,47.615605,-122.203824,"{'address1': '100 Bellevue Sq', 'address2': ''...","[100 Bellevue Sq, Bellevue, WA 98004]","100 Bellevue Sq\nBellevue, WA 98004"
833,LvUsQR3_dXoPySFQX6zlew,mae-ploy-thai-cuisine-seattle,Mae Ploy Thai Cuisine,https://s3-media1.fl.yelpcdn.com/bphoto/6P8S8z...,False,https://www.yelp.com/biz/mae-ploy-thai-cuisine...,130,"[{'alias': 'thai', 'title': 'Thai'}]",3.5,"['pickup', 'delivery']","{""address1"": ""6421 15th Ave NW"", ""address2"": ""...",1.206784e+10,(206) 784-0899,9246.128673,$$,47.675770,-122.376670,"{'address1': '6421 15th Ave NW', 'address2': '...","[6421 15th Ave NW, Seattle, WA 98107]","6421 15th Ave NW\nSeattle, WA 98107"
835,H_gsS9J920u_hK7jekobqw,suite-bellevue-4,Suite,https://s3-media1.fl.yelpcdn.com/bphoto/jdYMfv...,False,https://www.yelp.com/biz/suite-bellevue-4?adju...,225,"[{'alias': 'lounges', 'title': 'Lounges'}, {'a...",3.0,['delivery'],"{""address1"": 

### Final Map

In [96]:
## make ourn final map and save as varaible
pfig = px.scatter_mapbox(df, lat='latitude',lon='longitude',
                  mapbox_style='open-street-map', hover_name='name',
                         hover_data=['price','rating','Address'],
                 color='rating',
                        title='Restaurants with Pizza in Seattle, WA')
pfig.show()

#### HTML Uses `<br>` instead of `\n`

In [97]:
## apply the join to every row with a lambda
df['Address'] = df['display_address'].apply(lambda x: '<br>'.join(x))
df

/var/folders/td/cybzgn316hq8r5cn05lb0y2m0000gn/T/ipykernel_3854/1844012891.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,location,phone,display_phone,distance,price,latitude,longitude,display_location,display_address,Address
1,Wi6LFkjIausYj277ru6pqg,humble-pie-seattle,Humble Pie,https://s3-media4.fl.yelpcdn.com/bphoto/A08GOZ...,False,https://www.yelp.com/biz/humble-pie-seattle?ad...,350,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,['delivery'],"{""address1"": ""525 Rainier Ave S"", ""address2"": ...",1.206330e+10,(206) 329-5133,2166.834011,$$,47.597649,-122.313306,"{'address1': '525 Rainier Ave S', 'address2': ...","[525 Rainier Ave S, Seattle, WA 98144]","525 Rainier Ave S<br>Seattle, WA 98144"
4,M9xzvwgK58T0w7wvXedvuQ,hot-mamas-pizza-seattle,Hot Mama's Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/LEL1qj...,False,https://www.yelp.com/biz/hot-mamas-pizza-seatt...,937,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,['delivery'],"{""address1"": ""700 E Pine St"", ""address2"": """", ...",1.206323e+10,(206) 322-6444,2080.359405,$,47.615379,-122.323231,"{'address1': '700 E Pine St', 'address2': '', ...","[700 E Pine St, Seattle, WA 98122]","700 E Pine St<br>Seattle, WA 98122"
5,fxyHWmfzcdjImgQ_UYHoTw,bar-cotto-seattle,Bar Cotto,https://s3-media3.fl.yelpcdn.com/bphoto/07DoPF...,False,https://www.yelp.com/biz/bar-cotto-seattle?adj...,335,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.5,['delivery'],"{""address1"": ""1546 15th Ave"", ""address2"": """", ...",1.206839e+10,(206) 838-8081,1292.526796,$$,47.614693,-122.312764,"{'address1': '1546 15th Ave', 'address2': '', ...","[1546 15th Ave, Seattle, WA 98122]","1546 15th Ave<br>Seattle, WA 98122"
7,et4eWK49x5od4lfqCukG6w,central-pizza-seattle,Central Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/DE4G4e...,False,https://www.yelp.com/biz/central-pizza-seattle...,196,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.0,"['pickup', 'delivery']","{""address1"": ""2901 S Jackson St"", ""address2"": ...",1.206603e+10,(206) 602-6333,1571.867438,$$,47.598976,-122.294685,"{'address1': '2901 S Jackson St', 'address2': ...","[2901 S Jackson St, Seattle, WA 98144]","2901 S Jackson St<br>Seattle, WA 98144"
9,cAFXng3Gr1cBCFkP1VIxng,the-independent-pizzeria-seattle-2,The Independent Pizzeria,https://s3-media4.fl.yelpcdn.com/bphoto/4gIuis...,False,https://www.yelp.com/biz/the-independent-pizze...,179,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,['delivery'],"{""address1"": ""4235 E Madison St"", ""address2"": ...",1.206861e+10,(206) 860-6110,2897.252844,$$,47.636044,-122.277377,"{'address1': '4235 E Madison St', 'address2': ...","[4235 E Madison St, Seattle, WA 98112]","4235 E Madison St<br>Seattle, WA 98112"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
832,IelbaEgtvDbAFucJWLUWRw,marketplace-cafe-bellevue-7,Marketplace Cafe,https://s3-media1.fl.yelpcdn.com/bphoto/JxhPMe...,False,https://www.yelp.com/biz/marketplace-cafe-bell...,126,"[{'alias': 'newamerican', 'title': 'American (...",3.0,['delivery'],"{""address1"": ""100 Bellevue Sq"", ""address2"": """"...",1.425647e+10,(425) 646-5006,6891.374266,$$,47.615605,-122.203824,"{'address1': '100 Bellevue Sq', 'address2': ''...","[100 Bellevue Sq, Bellevue, WA 98004]","100 Bellevue Sq<br>Bellevue, WA 98004"
833,LvUsQR3_dXoPySFQX6zlew,mae-ploy-thai-cuisine-seattle,Mae Ploy Thai Cuisine,https://s3-media1.fl.yelpcdn.com/bphoto/6P8S8z...,False,https://www.yelp.com/biz/mae-ploy-thai-cuisine...,130,"[{'alias': 'thai', 'title': 'Thai'}]",3.5,"['pickup', 'delivery']","{""address1"": ""6421 15th Ave NW"", ""address2"": ""...",1.206784e+10,(206) 784-0899,9246.128673,$$,47.675770,-122.376670,"{'address1': '6421 15th Ave NW', 'address2': '...","[6421 15th Ave NW, Seattle, WA 98107]","6421 15th Ave NW<br>Seattle, WA 98107"
835,H_gsS9J920u_hK7jekobqw,suite-bellevue-4,Suite,https://s3-media1.fl.yelpcdn.com/bphoto/jdYMfv...,False,https://www.yelp.com/biz/suite-bellevue-4?adju...,225,"[{'alias': 'lounges', 'title': 'Lounges'}, {'a...",3.0,['delivery'],"{

In [99]:
## make ourn final map and save as varaible
pfig = px.scatter_mapbox(df, lat='latitude',lon='longitude',
                  mapbox_style='open-street-map', hover_name='name',
                         hover_data=['price','rating','Address'],
                 color='rating',
                        title='Restaurants with Pizza in Seattle, WA')
pfig.show()

In [100]:
## use fig.write_html to save map
pfig.write_html('final_map.html')